**Μέλη Ομάδας**
- Αποστολάτος Ιωάννης sdi1900012
- Βασιλείου Ρηγίνος sdi1900019

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from datetime import date, timedelta
import datetime
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans

Open file

In [ ]:
filepath = '/content/drive/MyDrive/marketing_campaign.csv'

df = pd.read_csv(filepath, sep='\t')
df.head(5)

In [ ]:
df.info()

## Ερωτήματα 1


### Κάνουμε drop τα null values

In [ ]:
# find number of null values
df.isna().sum()

df = df.dropna()

### Μετατροπή date σε DateType objects

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')

today = date.today()
today = today.strftime('%d-%m-%Y')
today = pd.to_datetime(today, format='%d-%m-%Y')

df['Recency'] = df['Recency'].apply(lambda x: today - timedelta(days = x))

df.head()

## Ερώτημα 2

### Εκτυπώστε τις μοναδικές τιμές στα κατηγορικά χαρακτηριστικά Marital_Status και Education

In [ ]:
#get unique values of columns
unique1 = df['Education'].unique()
unique2 = df['Marital_Status'].unique()

print(unique1)
print(unique2)

### Αλλαγή alone, absurd, yolo σε single

In [ ]:
df['Marital_Status'] = df['Marital_Status'].replace('YOLO', 'Single')
df['Marital_Status'] = df['Marital_Status'].replace('Alone', 'Single')
df['Marital_Status'] = df['Marital_Status'].replace('Absurd', 'Single')

print(df['Marital_Status'].unique())

###Γραφήματα για πλήθος τιμών σε Marital status και Education

In [ ]:
marital_status_counts = df['Marital_Status'].value_counts()

marital_status_counts.plot(kind="bar")
plt.xlabel("categories")
plt.ylabel("count")
plt.title("Marital status")
plt.show()

In [ ]:
education_counts = df['Education'].value_counts()

education_counts.plot(kind="bar")
plt.xlabel("categories")
plt.ylabel("count")
plt.title("Education")
plt.show()

## Ερώτημα 3

### Α. Δημιουργία ενός χαρακτηριστικού ("Customer_For") που αντιπροσωπεύει τον αριθμό των ημερών που οι πελάτες άρχισαν να ψωνίζουν στο κατάστημα σε σχέση με την τελευταία καταγεγραμμένη ημερομηνία (Recency).


In [ ]:
# find recency date
df['Customer_For'] = df['Recency'] - df['Dt_Customer']

df.head()

### Β. Εξαγωγή της ηλικίας "Age" ενός πελάτη με βάση το "Year_Birth" που υποδεικνύει το έτος γέννησης του αντίστοιχου ατόμου.


In [ ]:
year = datetime.date.today().year

df['Age'] = year - df['Year_Birth']

df.head()

### Γ. Δημιουργήστε ένα άλλο χαρακτηριστικό "Spent" που υποδεικνύει το συνολικό ποσό που ξόδεψε ο πελάτης σε όλες τις κατηγορίες σε διάστημα δύο ετών

In [ ]:
df['Spent'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

df.head(10)

### Δ. Δημιουργήστε ένα χαρακτηριστικό "Παιδιά" για να δηλώσετε το σύνολο των παιδιών σε ένα νοικοκυριό, δηλαδή τα παιδιά και τους εφήβους.


In [ ]:
df['Children'] = df[['Kidhome', 'Teenhome']].sum(axis=1)

df.head(10)

### Ε. Για να αποκτήσετε περαιτέρω σαφήνεια του νοικοκυριού, δημιουργήστε ένα χαρακτηριστικό με ένδειξη "Family_Size" που δείχνει το συνολικό αριθμό των ατόμων σε ένα νοικοκυριό

In [ ]:
df['Family_Size'] = df['Children'] + df['Marital_Status'].apply(lambda x: 1 if (x == 'Single' or x == 'Divorced' or x == 'Widowed') else 2)

df['Family_Size'].head(10)

### Στ. Δημιουργήστε ένα χαρακτηριστικό "Is_Parent" που δηλώνει αν ένας πελάτης είναι και γονιός

In [ ]:
df['Is_Parent'] = df['Children'].apply(lambda x: 0 if (x == 0) else 1)

df['Is_Parent'].head(10)

### Ζ. Δημιουργήστε ένα άλλο χαρακτηριστικό "Living_With" χρησιμοποιώντας το "Marital_Status" για να εξάγετε την κατάσταση διαβίωσης των ζευγαριών. Συγκεκριμένα το χαρακτηριστικό αυτό πρέπει να έχει δύο τιμές, "Partner" και "Alone".


In [ ]:
df['Living_With'] = df['Marital_Status'].apply(lambda x: 'Alone' if (x == 'Single' or x == 'Divorced' or x == 'Widowed') else 'Partner')

df['Living_With'].head(10)

### Η. Δημιουργήστε τη στήλη “Age Group” χρησιμοποιώντας τη στήλη “Age”, η οποία να ομαδοποιεί τις ηλικίες στις παρακάτω τιμές "21-30", "31-40", "41-50", "51-60", "61-70", "71-80", ">80".

In [ ]:
#create bins and labels for each category
bins = [20, 30, 40, 50, 60, 70, 80, float('inf')]
labels = ['21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '>80']

df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels)

df['Age_Group'] = df['Age_Group'].astype(str)

df['Age_Group'].head(10)

## Ερώτημα 4.

### Ελέγξτε αν υπάρχουν ακραίες τιμές στα χαρακτηριστικά και διαγράψτε τις από τα δεδομένα

In [ ]:
df.describe()

###Αφαίρεση ακραίων τιμών από Year Birth

In [ ]:
threshold = 0.001
removed_rows = df[df['Year_Birth'] <= df['Year_Birth'].quantile(threshold)]
df = df[df['Year_Birth'] > df['Year_Birth'].quantile(threshold)]

print(removed_rows)
df.shape

###Αφαίρεση ακραίων τιμών από Income

In [ ]:
threshold = 0.1
removed_rows = df[df['Income'] <= df['Income'].quantile(threshold)]
df = df[df['Income'] > df['Income'].quantile(threshold)]

threshold = 0.99
removed_rows = df[df['Income'] >= df['Income'].quantile(threshold)]
df = df[df['Income'] < df['Income'].quantile(threshold)]

print(removed_rows)
df.shape

###Αφαίρεση ακραίων τιμών από Spent

In [ ]:
threshold = 0.01
removed_rows = df[df['Spent'] <= df['Spent'].quantile(threshold)]
df = df[df['Spent'] > df['Spent'].quantile(threshold)]

print(removed_rows)
df.shape

## Ερώτημα 5

### Στη συνέχεια, εξετάστε τη συσχέτιση μεταξύ των χαρακτηριστικών με ένα heatmap διάγραμμα. (Εξαιρώντας τα κατηγορικά χαρακτηριστικά σε αυτό το σημείο)

In [ ]:
df.info()

### Αφαίρεση non-numerical data και δεδομένων που υπάρχουν σε άλλη κατηγορία πχ MntWines στο Spent

In [ ]:
# select only the columns with numerical data
new_df = df.copy()

new_df = new_df.drop('Age_Group', axis=1)
new_df = new_df.drop('Living_With', axis=1)
new_df = new_df.drop('Is_Parent', axis=1)
new_df = new_df.drop('Customer_For', axis=1)
new_df = new_df.drop('Recency', axis=1)
new_df = new_df.drop('Dt_Customer', axis=1)
new_df = new_df.drop('MntWines', axis=1)
new_df = new_df.drop('MntFruits', axis=1)
new_df = new_df.drop('MntFishProducts', axis=1)
new_df = new_df.drop('MntMeatProducts', axis=1)
new_df = new_df.drop('MntSweetProducts', axis=1)
new_df = new_df.drop('MntGoldProds', axis=1)
new_df = new_df.drop('Education', axis=1)
new_df = new_df.drop('Marital_Status', axis=1)
new_df = new_df.drop('ID', axis=1)
new_df = new_df.drop('Year_Birth', axis=1)
new_df = new_df.drop('Z_CostContact', axis=1)
new_df = new_df.drop('Z_Revenue', axis=1)

new_df

In [ ]:
plt.figure(figsize=(18, 14))

# Create the heatmap using seaborn
heatmap = sns.heatmap(new_df.corr(), annot= True, cmap='coolwarm')
cbar = heatmap.collections[0].colorbar  # Add a color bar for the color map
cbar.ax.set_ylabel('Value')  # Label the color bar

# Show the plot
plt.show()

## Ερώτημα 6

In [ ]:
def plot_bar(x_label, y_label, title, count):
  # Create the bar plot and set custom colors
  ax = count.plot(kind='bar', x=x_label, y=y_label, color=['r', 'g', 'b', 'y', 'black'])

  # Set the plot title and labels
  ax.set_title(title)
  ax.set_xlabel(x_label)
  ax.set_ylabel(y_label)

  # Show the plot
  plt.show()

####1. Σε ποιά κατηγορία Marital_Status ανήκει το μεγαλύτερο ποσοστό των πελατών της εταιρείας

In [ ]:
marital_status_counts = df['Marital_Status'].value_counts()

plot_bar('Marital Status', 'Count', 'Marital Status', marital_status_counts)

####2. Πόσοι πελάτες έχουν κάνει Complain ;

In [ ]:
plot_bar('Complain', 'Count', 'Complaints', df['Complain'].value_counts())

Παρατηρούμε ότι οι περισσότεροι πελάτες δεν έχουν κάνει complain

####3. Σχέση μεταξύ του αριθμού των αγορών Spent και της οικογενειακής κατάστασης.

In [ ]:
plot_bar('Marital Status', 'Spent', 'Marital Status vs Spent', df.groupby('Marital_Status')['Spent'].mean())

####5. Τι σχέση έχει η ηλικία Age Group με το χαρακτηριστικό Spent των αγορών;

In [ ]:
plot_bar('Age_Group', 'Spent', 'Age Group vs Spent', df.groupby('Age_Group')['Spent'].mean())

####6. Τι σχέση έχει το εισόδημα Income με το χαρακτηριστικό Spent των αγορών;

In [ ]:
df.plot(kind='scatter', x='Income', y='Spent')

# Set the plot title and labels
plt.title('Income vs. Spent')
plt.xlabel('Income')
plt.ylabel('Spent')

# Show the plot
plt.show()

####7. Ποια είναι η σχέση μεταξύ της εκπαίδευσης και του εισοδήματος;

In [ ]:
plot_bar('Education', 'Income', 'Income - Education', df.groupby('Education')['Income'].mean())

####8. Ποια είναι η σχέση μεταξύ του εισοδήματος και του μεγέθους της οικογένειας;

In [ ]:
plot_bar('Family_Size', 'Income', 'Income - Family Size', df.groupby('Family_Size')['Income'].mean())

####9. Ποια είναι η σχέση μεταξύ του εισοδήματος και του αριθμού των παιδιών;

In [ ]:
plot_bar('Children', 'Income', 'Income -Children', df.groupby('Children')['Income'].mean())

####10. Ποια είναι η σχέση μεταξύ του εισοδήματος και του Living_With;

In [ ]:
plot_bar('Living_With', 'Income', 'Income -Living_With', df.groupby('Living_With')['Income'].mean())

####15. Σχεδιάστε το ιστόγραμμα για τη στήλη Income.

In [ ]:
df['Income'].hist()

plt.title('Income Histogram')
plt.xlabel('Income')
plt.ylabel('Frequency')

plt.show()

####16. Σχεδιάστε το ιστόγραμμα για τη στήλη Kidhome.

In [ ]:
df['Kidhome'].hist()

# Set the plot title and labels
plt.title('Histogram of Kidhome')
plt.xlabel('Kids home')
plt.ylabel('Frequency')

# Show the plot
plt.show()

## 7

###Label Encoder μετατροπή κατηγοριών σε αριθμητικές τιμές

In [ ]:
# create instance of label encoder
labelencoder = LabelEncoder()

df['Marital_Status_Encoding'] =  labelencoder.fit_transform(df['Marital_Status'])
df['Education_Encoding'] =  labelencoder.fit_transform(df['Education'])
df['Living_With_Encoding'] =  labelencoder.fit_transform(df['Living_With'])

df.head(10)

###Δημιουργία αντιγράφου dataframe

In [ ]:
num_cols = df.select_dtypes(include=['float64', 'int64'])

new_df = num_cols.copy()

new_df.head(10)

###Διαγραφή στηλών που σχετίζονται με προσφορές και προωθητικές
ενέργειες

In [ ]:
new_df = new_df.drop(['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
'AcceptedCmp2', 'Complain', 'Response'], axis=1)

new_df.head(10)

###Standardization

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(new_df)

scaled_data

###Χρήση της PCA για μείωση διαστάσεων σε n=3

In [ ]:
pca = PCA(n_components=3)

X_pca = pca.fit_transform(scaled_data)

X_pca

###Σχεδιάστε την (τρισδιάστατη) προβολή του αποτελέσματος

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

x = X_pca[:, 0]
y = X_pca[:, 1]
z = X_pca[:, 2]

ax.scatter(x, y, z, s=50)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

plt.show()

##8

###Μέθοδος ELBOW για τον προσδιορισμό του αριθμού των συστάδων που πρέπει να σχηματιστούν

In [ ]:
# Calculate the score for different K-mean values
sse = []
for k in range(1, 11):
  kmeans = KMeans(n_clusters=k, n_init=10)
  kmeans.fit(scaled_data)
  sse.append(kmeans.inertia_)

# Plot the data
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

###Agglomerative και K-Means

In [ ]:
# Agglomerative Clustering
agg = AgglomerativeClustering(n_clusters=3, linkage='ward')
agg_labels = agg.fit_predict(X_pca)

# K-Means Clustering
kmeans = KMeans(n_clusters=3, n_init=10)
kmeans_labels = kmeans.fit_predict(X_pca)

# Plot the clusters
fig = plt.figure(figsize=(15, 6))

# Agglomerative Clustering plot
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=agg_labels)
ax1.set_title('Agglomerative Clustering')

# K-Means Clustering plot
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=kmeans_labels)
ax2.set_title('K-Means Clustering')

plt.show()

##9

In [ ]:
new_df['cluster_labels'] = kmeans_labels

cluster_stats = new_df.groupby('cluster_labels').agg(['mean', 'median', 'std'])

# Print the basic characteristics of each cluster
cluster_stats['NumWebVisitsMonth']

Συγκρίνοντας τα στατιστικά κάθε cluster βγάλαμε τα εξής συμπεράσματα :

cluster 0 :
* μεγαλύτεροι σε ηλικία
* οι περισσότεροι είναι γονείς
* έχουν κάνει τις περισσότερες αγορές με discount

cluster 1 :
* έχουν το περισσότερο εισόδημα
* οι περισσότεροι δεν είναι γονείς
* ξοδεύουν περισσότερα
* έχουν κάνει τις λιγότερες αγορές με discount
* έχει τις λιγότερες επισκέψεις online

cluster 2 :
* έχουν τουλάχιστον 3μελη οικογένεια
* έχουν το μικρότερο εισόδημα
* είναι οι πιο νέοι σε ηλικία
* ξοδεύουν λιγότερα
* έχει τις περισσότερες επισκέψεις online


